# Compare NLP Techniques: Build A Basic RNN

### Read In Cleaned Text

In [80]:
# Load the cleaned training and test sets
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Performed my data cleaning separately.

I have done the cleaning to the NARRATIVE column on both my train and test datasets and saved the train_x, train_y, test_x, test_y and testd datasets.
And uploaded them directly here below from my computer.

In [98]:
X_train.to_csv('/Users/mrudulareddy/Desktop/submissionR/X_trai.csv', index=False, header=True)
X_test.to_csv('/Users/mrudulareddy/Desktop/submissionR/X_tes.csv', index=False, header=True)
y_train.to_csv('/Users/mrudulareddy/Desktop/submissionR/y_trai.csv', index=False, header=True)
y_test.to_csv('/Users/mrudulareddy/Desktop/submissionR/y_tes.csv', index=False, header=True)

In [99]:
X_train = pd.read_csv('/Users/mrudulareddy/Desktop/submissionR/X_trai.csv')
X_test = pd.read_csv('/Users/mrudulareddy/Desktop/submissionR/X_tes.csv')
y_train = pd.read_csv('/Users/mrudulareddy/Desktop/submissionR/y_trai.csv')
y_test = pd.read_csv('/Users/mrudulareddy/Desktop/submissionR/y_tes.csv')


In [100]:
testd.head()

,clean_text
0,"['unk', 'susp', 'smashed', 'vic', 'front', 'dr..."
1,"['unk', 'susps', 'used', 'unk', 'object', 'pry..."
2,['nan']
3,"['unk', 'susp', 'smashed', 'victs', 'veh', 'wi..."
4,"['unk', 'susp', 'cut', 'vic', 'door', 'screw',..."


### Prep Data

In [101]:
X_train.head()

,id,NARRATIVE,clean_text
0,183896,UNK SUSPS PRYED OFF SECURITY BARS FROM FT UPPE...,"['unk', 'susps', 'pryed', 'security', 'bars', ..."
1,2478,UNK SUSP RMVD FOG LIGHTS FROM VICTS VEH AND FL...,"['unk', 'susp', 'rmvd', 'fog', 'lights', 'vict..."
2,12332,NaN,['nan']
3,145123,UNK S GAINED ENTRY INTO V VEH THRU PASS FRONT...,"['', 'unk', 'gained', 'entry', 'v', 'veh', 'th..."
4,84919,UNK S USED UNK HARD OBJECT TO SMASH VICTS VEH ...,"['unk', 'used', 'unk', 'hard', 'object', 'smas..."


In [102]:
testd.head()

,clean_text
0,"['unk', 'susp', 'smashed', 'vic', 'front', 'dr..."
1,"['unk', 'susps', 'used', 'unk', 'object', 'pry..."
2,['nan']
3,"['unk', 'susp', 'smashed', 'victs', 'veh', 'wi..."
4,"['unk', 'susp', 'cut', 'vic', 'door', 'screw',..."


In [103]:
# Train the tokenizer and use that tokenizer to convert the sentences to sequences of numbers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['clean_text'])
X_train_seq = tokenizer.texts_to_sequences(X_train['clean_text'])
X_test_seq = tokenizer.texts_to_sequences(X_test['clean_text'])
test_seq = tokenizer.texts_to_sequences(testd['clean_text'])
test_seq_padded = pad_sequences(test_seq, 50)
# Pad the sequences so each sequence is the same length
X_train_seq_padded = pad_sequences(X_train_seq, 50)
X_test_seq_padded = pad_sequences(X_test_seq, 50)

### Build And Evaluate RNN

In [106]:
# Import the tools needed and use our previously defined functions to calculate precision and recall
import keras.backend as K
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
# Construct our basic RNN model framework
model = Sequential()

model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(32, dropout=0, recurrent_dropout=0))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', precision_m, recall_m])
# Fit the RNN
history = model.fit(X_train_seq_padded, y_train['CRIMETYPE'], 
                    batch_size=32, epochs=2,
                    validation_data=(X_test_seq_padded, y_test))
test_acc = model.evaluate(X_test_seq_padded)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          695744    
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 32)                1056      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 705,153
Trainable params: 705,153
Non-trainable params: 0
_________________________________________________________________


Epoch 1/2
3125/3125 [==============================] - 48s 15ms/step - loss: 0.0049 - accuracy: 0.9998 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 3.4963e-06 - val_accuracy: 1.0000 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/2
3125/3125 [==============================] - 46s 15ms/step - loss: 1.3986e-06 - accuracy: 1.0000 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 4.3321e-07 - val_accuracy: 1.0000 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


In [ ]:
predictions = model.predict(np.array([sample_text]))

3125/3125 [==============================] - 3s 857us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
